In [3]:
from functools import partial
from keras.initializers import he_uniform
from keras.backend import int_shape, squeeze
from keras.layers.convolutional_recurrent import ConvLSTM2D
import numpy as np
import os
from keras.layers import Input, Reshape, Lambda
from math import ceil
from functools import partial
from keras.backend import int_shape, squeeze
import tensorflow as tf
from keras.layers import Input
import librosa

In [6]:
def listdirInMac(path):
    os_list = os.listdir(path)
    for item in os_list:
        if item.startswith('.') and os.path.isfile(os.path.join(path, item)):
            os_list.remove(item)
    return os_list

def traversalDir_FirstDir(path):
    dict = {}
    files = listdirInMac(path)
    for file in files:
        m = os.path.join(path, file)
        h = os.path.split(m)
        dict[h[1]] = []
        song_wav = listdirInMac(m)
        m = m + '/'
        for track in song_wav:
            value = os.path.join(m, track)
            dict[h[1]].append(value)
    return dict

mix_path = traversalDir_FirstDir('DSD100subset/Mixtures/Dev')

sou_path = traversalDir_FirstDir('DSD100subset/Sources/Dev')

all_path = mix_path.copy()
for key in all_path.keys():
    all_path[key].extend(sou_path[key])

print("%d mixtures in the given path to train"%len(mix_path))
print(all_path)

2 mixtures in the given path to train
{"055 - Angels In Amplifiers - I'm Alright": ["DSD100subset/Mixtures/Dev\\055 - Angels In Amplifiers - I'm Alright/mixture.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/bass.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/drums.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/other.wav", "DSD100subset/Sources/Dev\\055 - Angels In Amplifiers - I'm Alright/vocals.wav"], '081 - Patrick Talbot - Set Me Free': ['DSD100subset/Mixtures/Dev\\081 - Patrick Talbot - Set Me Free/mixture.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/bass.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/drums.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/other.wav', 'DSD100subset/Sources/Dev\\081 - Patrick Talbot - Set Me Free/vocals.wav']}


In [7]:
def loadAudioFile(filePath):
    audio, sampleRate = librosa.load(filePath, sr = 44100)  # Load an audio file as a floating point time series， 默认采样率sr=22050
    return audio, sampleRate

# Return a 2d numpy array of the spectrogram
def audioFileToSpectrogram(audioFile, fftWindowSize = 1024):
    # time resolution N/fs=1024/44100=23ms
    spectrogram = librosa.stft(audioFile, fftWindowSize)   # 返回复数值矩阵D , STFT矩阵D中的行数是（1 + 第二个参数一般2的幂n_fft / 2）
    phase = np.angle(spectrogram)
    amplitude = np.log1p(np.abs(spectrogram))   # np.abs(D[f, t]) is the magnitude of frequency bin f at frame帧 t， loge(1+ )
    return amplitude, phase

频谱矩阵做random time stretch

In [46]:
import tensorflow as tf

# audio/spectrogram.py

def time_stretch(
        spectrograms,
        factor=1.0,
        method=tf.image.ResizeMethod.BILINEAR):
    """ Time stretch a spectrogram preserving shape in tensorflow. Note that
    this is an approximation in the frequency domain.
    :param spectrogram: Input spectrogram to be time stretched as tensor.
    :param factor: (Optional) Time stretch factor, must be >0, default to 1.
    :param mehtod: (Optional) Interpolation method, default to BILINEAR.
    :returns: Time stretched spectrogram as tensor with same shape.
    """
    F = tf.shape(spectrograms)[0]
    T = tf.shape(spectrograms)[1]
    T_ts = tf.cast(tf.cast(T, tf.float32) * factor, tf.int32)[0]  #cast: 改数据类型
    print(T, T_ts, F)  # 7587, 6097, 513
    ts_spec = tf.image.resize(
        spectrograms,
        size = [F, T_ts],
        method=method)      # 双线性插值
    return tf.image.resize_with_crop_or_pad(ts_spec, F, T)   # 补/切回原来形状


def random_time_stretch(spectrogram, factor_min=0.9, factor_max=1.1, **kwargs):
    """ Time stretch a spectrogram preserving shape with random ratio in
    tensorflow. Applies time_stretch to spectrogram with a random ratio drawn
    uniformly in [factor_min, factor_max].
    :param spectrogram: Input spectrogram to be time stretched as tensor.
    :param factor_min: (Optional) Min time stretch factor, default to 0.9.
    :param factor_max: (Optional) Max time stretch factor, default to 1.1.
    :returns: Randomly time stretched spectrogram as tensor with same shape.
    """
    factor = tf.random.uniform(
        shape=(1,),
        seed=0) * (factor_max - factor_min) + factor_min
    return time_stretch(spectrogram, factor=factor, **kwargs)


频谱矩阵做random pitch shift

In [64]:
def pitch_shift(
        spectrograms,
        semitone_shift=0.0,
        method=tf.image.ResizeMethod.BILINEAR):
    """ Pitch shift a spectrogram preserving shape in tensorflow. Note that
    this is an approximation in the frequency domain.
    :param spectrogram: Input spectrogram to be pitch shifted as tensor.
    :param semitone_shift: (Optional) Pitch shift in semitone半音, default to 0.0.
    :param mehtod: (Optional) Interpolation method, default to BILINEAR.
    :returns: Pitch shifted spectrogram (same shape as spectrogram).
    """
    factor = 2 ** (semitone_shift / 12.)
    F = tf.shape(spectrograms)[0]
    T = tf.shape(spectrograms)[1]
    F_ps = tf.cast(tf.cast(F, tf.float32) * factor, tf.int32)[0]
    ps_spec = tf.image.resize(
        spectrograms,
        [F_ps,T],
        method=method)
    paddings = [[0, tf.maximum(0, F - F_ps)], [0, 0], [0, 0]]
    return tf.pad(ps_spec[:F,:,:], paddings, 'CONSTANT')


def random_pitch_shift(spectrogram, shift_min=-1., shift_max=1., **kwargs):
    """ Pitch shift a spectrogram preserving shape with random ratio in
    tensorflow. Applies pitch_shift to spectrogram with a random shift
    amount (expressed in semitones) drawn uniformly in [shift_min, shift_max].
    :param spectrogram: Input spectrogram to be pitch shifted as tensor.
    :param shift_min: (Optional) Min pitch shift in semitone, default to -1.
    :param shift_max: (Optional) Max pitch shift in semitone, default to 1.
    :returns: Randomly pitch shifted spectrogram (same shape as spectrogram).
    """
    semitone_shift = tf.random.uniform(
        shape=(1,),
        seed=0) * (shift_max - shift_min) + shift_min
    return pitch_shift(spectrogram, semitone_shift=semitone_shift, **kwargs)

In [67]:
def expandToGrid(spectrogram, time_scale, ratio_overlap):
    # 条件： time_scale * ratio_overlap 是整数
    K = int(ceil((spectrogram.shape[1] / time_scale - ratio_overlap)/(1-ratio_overlap)))  # ceil取上
    newY = int(time_scale + time_scale * (1-ratio_overlap) * (K-1))
    newX = spectrogram.shape[0]    # 行数不变
    newSpectrogram = np.zeros((newX, newY))   # 右：多一些0，至能被girdsize整除
    newSpectrogram[:, :spectrogram.shape[1]] = spectrogram
    return newSpectrogram

In [68]:
def chop(matrix, time_scale, ratio_overlap):
    slices = []
    for time in range(0, ceil((matrix.shape[1]-time_scale)/time_scale/(1-ratio_overlap))+1):   # 列
            s = matrix[:, int(time * time_scale*(1-ratio_overlap)) :int(time * time_scale*(1-ratio_overlap)+time_scale)] # 切为很多小块，每块513*scale
            s = np.transpose(s) # 每块大小：scale*513
            slices.append(s)  # 存到slices列表， 使得模型的输入都一样大
    slices_np = np.array(slices)
    print(slices_np.shape)
    return slices_np

In [70]:
def song2spectrogram(all_path_para, fftWindowSize=1024, time_scale=30, overlap_ratio=0.5):
    

    x = np.empty((0,30,513), float)
    x_phase = []
    y_bass = np.empty((0,30,513), float)
    y_drums = np.empty((0,30,513), float)
    y_other = np.empty((0,30,513), float)
    y_vocals = np.empty((0,30,513), float)
    # 第0维：每首歌
    # 第1维：height = f
    # 第2维：width = t
    # 第3维：每个音轨 - x,y_bass,y_drums,y_other,y_vocals 共5个
    for key in all_path_para:   # 每首歌
        path_list = all_path_para[key][:]
        audio, sampleRate = loadAudioFile(path_list[0])
        spectrogram, phase = audioFileToSpectrogram(audio, fftWindowSize)
        each_song = np.empty([int(spectrogram.shape[0]),int(spectrogram.shape[1]),0])
        for path in path_list:    # 每个音轨
            audio, sampleRate = loadAudioFile(path)           
            spectrogram, phase = audioFileToSpectrogram(audio, fftWindowSize)
            spectrogram = spectrogram[:,:,np.newaxis]            
            each_song = np.append(each_song, spectrogram, axis=-1)
        print('sample rate: ',sampleRate)
        print("spectrogram shape:",spectrogram.shape)
        print("each_song shape: ", each_song.shape)  # (5, 513, 7587)
        
        # 输入输出都做数据增强：time stretch，pitch swift
        ts_spectrogram = random_time_stretch(each_song)   
        print("Time stretch Spectrogram shape:",ts_spectrogram.shape)
        ps_spectrogram = random_pitch_shift(each_song)
        print("Pitch swift Spectrogram shape:",ps_spectrogram.shape)
                
        [hight, weidth, stem] = each_song.shape
        for spect in [each_song, ts_spectrogram, ps_spectrogram]:
            for i in range(stem):
                expandedSpectrogram = expandToGrid(spect[:,:,i],time_scale,overlap_ratio)
                slices = chop(expandedSpectrogram, time_scale,overlap_ratio)  
                if i==0:
                    x = np.append(x, slices,axis=0)
                    x_phase.append(phase)
                elif i==1:
                    y_bass = np.append(y_bass, slices,axis=0)
                elif i==2:
                    y_drums = np.append(y_drums, slices,axis=0)
                elif i==3:
                    y_other = np.append(y_other, slices,axis=0)
                else:   #i==4
                    y_vocals = np.append(y_vocals, slices,axis=0)
                
    output = [x,x_phase,y_bass,y_drums,y_other,y_vocals]
  
    return output

list_dataset = song2spectrogram(all_path_para=all_path)

sample rate:  44100
spectrogram shape: (513, 7587, 1)
each_song shape:  (513, 7587, 5)
tf.Tensor(7587, shape=(), dtype=int32) tf.Tensor(7310, shape=(), dtype=int32) tf.Tensor(513, shape=(), dtype=int32)
Time stretch Spectrogram shape: (513, 7587, 5)
Pitch swift Spectrogram shape: (513, 7587, 5)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
(505, 30, 513)
sample rate:  44100
spectrogram shape: (513, 7244, 1)
each_song shape:  (513, 7244, 5)
tf.Tensor(7244, shape=(), dtype=int32) tf.Tensor(7114, shape=(), dtype=int32) tf.Tensor(513, shape=(), dtype=int32)
Time stretch Spectrogram shape: (513, 7244, 5)
Pitch swift Spectrogram shape: (513, 7244, 5)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(482, 30, 513)
(48

In [71]:
list_dataset[0].shape

(2961, 30, 513)